In [2]:
import ipywidgets as widgets
print("#1 insert client data: token, app_secret, part list")
tbToken = widgets.Text(
    placeholder='Anonymous key here (token)',
    description='token =',
    value='b64100390e3d1cf5deb5389147e91f64e85da13a37cb3'
)

tbAppSecret = widgets.Text(
    placeholder='Application secret: here (app_secret)',
    description='app_secret =',
    value='44dc34e355e9cd744949'
)

tbPartList = widgets.Textarea(
    placeholder='Part list (new line separated)',
    description='part list =',
    value='NE555D\n1N4007-DC\nBTM-222'
)

display(tbToken)
display(tbAppSecret)
display(tbPartList)

#1 insert client data: token, app_secret, part list


Text(value='b64100390e3d1cf5deb5389147e91f64e85da13a37cb3', description='token =', placeholder='Anonymous key …

Text(value='44dc34e355e9cd744949', description='app_secret =', placeholder='Application secret: here (app_secr…

Textarea(value='NE555D\n1N4007-DC\nBTM-222', description='part list =', placeholder='Part list (new line separ…

In [3]:
import json
print("#2 parse part list")
s = tbPartList.value
params = {}
for item in s.split("\n"):
    params["SymbolList["+str(len(params))+"]" ] = item
print(json.dumps(params, indent=4, sort_keys=True))

#2 parse part list
{
    "SymbolList[0]": "NE555D",
    "SymbolList[1]": "1N4007-DC",
    "SymbolList[2]": "BTM-222"
}


In [4]:
print("#3 add others parameters to dict")
params['Country']='PL'
params['Language']='PL'
params['Currency']='PLN' ### new!
params


#3 add others parameters to dict


{'SymbolList[0]': 'NE555D',
 'SymbolList[1]': '1N4007-DC',
 'SymbolList[2]': 'BTM-222',
 'Country': 'PL',
 'Language': 'PL',
 'Currency': 'PLN'}

In [5]:
print("#4 prepare signature_base (for api_signature)")
# add token
params['Token']=tbToken.value
# sort all
from collections import OrderedDict
params = OrderedDict(sorted(params.items()));
# encode params
import urllib.parse
encoded_params = urllib.parse.urlencode(params, '')
# api addr
api_url = 'https://api.tme.eu/Products/GetStocks.json'
import urllib.parse
signature_base = 'POST' + '&' + urllib.parse.quote(api_url, '') + '&' + urllib.parse.quote(encoded_params, '')
# print(signature_base)

#4 prepare signature_base (for api_signature)


In [6]:
print("#5 generate api_signature (input data: signature_base, app_secret)")
import base64, hmac, hashlib
app_secret_bytes = str.encode(tbAppSecret.value)
signature_base_bytes = str.encode(signature_base)
api_signature = base64.encodebytes(hmac.new(app_secret_bytes, signature_base_bytes, hashlib.sha1).digest()).rstrip()
# print(api_signature)

#5 generate api_signature (input data: signature_base, app_secret)


In [7]:
print("#6 add api_signature to params and send HTTP request")
import requests, json

params['ApiSignature'] = api_signature

headers = {
    'Content-Type': "application/x-www-form-urlencoded"
    }

response = requests.request("POST", api_url, data=params, headers=headers)
response.text

#6 add api_signature to params and send HTTP request


'{"Status":"OK","Data":{"ProductList":[{"Symbol":"1N4007-DC","Amount":1736760,"Unit":"szt"},{"Symbol":"BTM-222","Amount":0,"Unit":"szt"},{"Symbol":"NE555D","Amount":14618,"Unit":"szt"}]}}'

In [8]:
print("#7 extrad necessary data from HTTP respond")
extracted_data = json.loads(response.text)
extracted_data = extracted_data['Data']
extracted_data = extracted_data['ProductList']
print(json.dumps(extracted_data, indent=4, sort_keys=True))

#7 extrad necessary data from HTTP respond
[
    {
        "Amount": 1736760,
        "Symbol": "1N4007-DC",
        "Unit": "szt"
    },
    {
        "Amount": 0,
        "Symbol": "BTM-222",
        "Unit": "szt"
    },
    {
        "Amount": 14618,
        "Symbol": "NE555D",
        "Unit": "szt"
    }
]


In [9]:
print("#8 data presentation")
from pandas.io.json import json_normalize
json_normalize(extracted_data)

#8 data presentation


,Amount,Symbol,Unit
0,1736760,1N4007-DC,szt
1,0,BTM-222,szt
2,14618,NE555D,szt
